### Setup

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
import argparse
from pprint import pprint
from time import sleep
from context import Context
from contracts.farm_contract import FarmContract
from contracts.staking_contract import StakingContract
from contracts.metastaking_contract import MetaStakingContract
from contracts.unstaker_contract import UnstakerContract
from utils.utils_generic import get_file_from_url_or_path, get_continue_confirmation
from utils.utils_chain import get_bytecode_codehash, base64_to_hex, WrapperAddress
from tools.runners.farm_runner import upgrade_farmv2_contract
from tools.runners.staking_runner import upgrade_staking_contracts
from tools.runners.metastaking_runner import upgrade_metastaking_contract
from tools.runners.generic_runner import upgrade_generic_contract

config.DEFAULT_PROXY = "https://gateway.multiversx.com"
config.DEFAULT_API = "https://api.multiversx.com"
config.DEFAULT_OWNER = config.DEFAULT_WORKSPACE.absolute() / "wallets" / "C1.pem"
config.DEFAULT_ADMIN = config.DEFAULT_WORKSPACE.absolute() / "wallets" / "C1.pem"

config.DEFAULT_CONFIG_SAVE_PATH = config.DEFAULT_WORKSPACE.absolute() / "deploy" / "configs-mainnet"

context = Context()

In [ ]:
farm_contract = FarmContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqr9gx79xls33tnzt0a94uvnkf452fz26q2jpsau4ulw")
staking_contract = StakingContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqqsx29p3fge7upkgup4mm5xsdsv4w7rh82jpsdcvdrt")
metastaking_contract = MetaStakingContract.load_contract_by_address("erd1qqqqqqqqqqqqqpgqxjdlk9d8gap8q2ghey6q5cg69y7p4rkd2jpsv89u3n")
unstaker_contract: UnstakerContract = context.get_contracts(config.UNSTAKERS)[0]

print("Farm contract:")
pprint(farm_contract.get_config_dict())
print("Staking contract:")
pprint(staking_contract.get_config_dict())
print("MetaStaking contract:")
pprint(metastaking_contract.get_config_dict())
print("Unstaker contract:")
pprint(unstaker_contract.get_config_dict())

### Operations

Farm pause

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = farm_contract.pause(context.deployer_account, context.network_provider.proxy)

Staking pause

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = staking_contract.pause(context.deployer_account, context.network_provider.proxy)

Farm upgrade

In [ ]:
farm_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/farm-with-locked-rewards.wasm"
expected_code_hash = "c1f2a3652371510767f5d64a4df02a3eda2744d54e9d91f7680dbf1bd02d82bb"

args = argparse.Namespace(address=farm_contract.address, bytecode=farm_bytecode_path, compare_states=True, all=False)
upgrade_farmv2_contract(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Staking upgrade

In [ ]:
staking_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/farm-staking.wasm"
expected_code_hash = "7eddcb67bef25c4a0c7de0c3e1b2444cbf84e05546ac61c29726b568f9efffde"

args = argparse.Namespace(address=staking_contract.address, bytecode=staking_bytecode_path, compare_states=True, all=False)
upgrade_staking_contracts(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(staking_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Metastaking upgrade

In [ ]:
metastaking_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/farm-staking-proxy.wasm"
expected_code_hash = "8a605fa8539f193a49d267ce73f9010904796d6b930ea7e4e09f9b4d9da1af9f"

args = argparse.Namespace(address=metastaking_contract.address, bytecode=metastaking_bytecode_path, compare_states=True, all=False)
upgrade_metastaking_contract(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(metastaking_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Unstaker upgrade

In [ ]:
unstaker_bytecode_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.4/token-unstake.wasm"
expected_code_hash = "b22c402e39ba46d6b0521d51e99e51c277b7cbf87c3795953754c084320c6300"

args = argparse.Namespace(address=unstaker_contract.address, bytecode=unstaker_bytecode_path, compare_states=True, all=False)
upgrade_generic_contract(args)

code_hash = context.network_provider.proxy.get_account(WrapperAddress(unstaker_contract.address)).code_hash
assert base64_to_hex(code_hash) == expected_code_hash

Farm resume

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = farm_contract.resume(context.deployer_account, context.network_provider.proxy)

Staking resume

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = staking_contract.resume(context.deployer_account, context.network_provider.proxy)

Unstaker burn fees reduction

In [ ]:
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = unstaker_contract.set_fees_burn_percentage(context.deployer_account, context.network_provider.proxy, [0])